In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import json
import re
import nltk
import tkinter as tk
from tkinter import ttk

In [2]:
movies_df = pd.read_csv("source_data/movies_metadata.csv")


C:\Users\chanc\AppData\Local\Temp\ipykernel_15856\721637515.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("source_data/movies_metadata.csv")


In [6]:
# Initialize the main window
root = tk.Tk()
root.title("Movie Recommender")

# Define the function to show recommendations
def show_recommendations():
    genre = genre_input.get()
    year = year_input.get()
    rating = rating_input.get()
    recommendations = recommend_movies(genre, year, rating)
    result_var.set("\n".join(recommendations))

# Simple recommendation function considering multiple inputs
def recommend_movies(genre, year, rating):
    filtered_movies = movies_df
    
    if genre:
        filtered_movies = filtered_movies[filtered_movies['genres'].str.contains(genre, case=False, na=False)]
    if year:
        filtered_movies = filtered_movies[filtered_movies['release_date'].str.contains(year, na=False)]
    if rating:
        filtered_movies = filtered_movies[filtered_movies['vote_average'] >= float(rating)]
    
    top_movies = filtered_movies.head(10)['title'].tolist()
    return top_movies

# Add a label and entry for genre input
ttk.Label(root, text="Enter your preferred genre:").grid(column=0, row=0, padx=10, pady=10)
genre_input = ttk.Entry(root)
genre_input.grid(column=1, row=0, padx=10, pady=10)

# Add a label and entry for year input
ttk.Label(root, text="Enter release year:").grid(column=0, row=1, padx=10, pady=10)
year_input = ttk.Entry(root)
year_input.grid(column=1, row=1, padx=10, pady=10)

# Add a label and entry for rating input
ttk.Label(root, text="Enter minimum rating:").grid(column=0, row=2, padx=10, pady=10)
rating_input = ttk.Entry(root)
rating_input.grid(column=1, row=2, padx=10, pady=10)

# Add a button to trigger recommendation
ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Add a label to display the results
result_var = tk.StringVar()
ttk.Label(root, textvariable=result_var).grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()
